<a href="https://colab.research.google.com/github/danielsoy/ALOCC-CVPR2018/blob/master/cuatro.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import models, transforms
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score, f1_score
import os
from PIL import Image
import numpy as np

Set random seed for reproducibility

In [3]:
torch.manual_seed(42)
np.random.seed(42)

Device configuration (use GPU if available)

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Custom Dataset Class

In [5]:
class CustomDataset(Dataset):
    def __init__(self, image_paths, labels, transform=None):
        self.image_paths = image_paths
        self.labels = labels
        self.transform = transform
    def __len__(self):
        return len(self.image_paths)
    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert("RGB")
        label = self.labels[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

Data Preprocessing

In [6]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # EfficientNet-B0 expects 224x224 images
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # ImageNet normalization
])

Load Dataset

In [7]:
def load_dataset(data_dir):
    image_paths = []
    labels = []

    # Assuming data_dir has two subfolders: "normal" and "anomalous"
    for label, class_name in enumerate(["normal", "anomalous"]):
        class_dir = os.path.join(data_dir, class_name)
        for image_name in os.listdir(class_dir):
            image_path = os.path.join(class_dir, image_name)
            image_paths.append(image_path)
            labels.append(label)
    return image_paths, labels

Split dataset into train, validation, and test sets

In [9]:
data_dir = "/content/drive/MyDrive/dataset tres"  # Replace with your dataset path
image_paths, labels = load_dataset(data_dir)
train_paths, test_paths, train_labels, test_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)
train_paths, val_paths, train_labels, val_labels = train_test_split(train_paths, train_labels, test_size=0.1, random_state=42)

Create DataLoader

In [10]:
train_dataset = CustomDataset(train_paths, train_labels, transform=transform)
val_dataset = CustomDataset(val_paths, val_labels, transform=transform)
test_dataset = CustomDataset(test_paths, test_labels, transform=transform)

In [11]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

Define the Model

In [12]:
class EfficientNetBinaryClassifier(nn.Module):
    def __init__(self):
        super(EfficientNetBinaryClassifier, self).__init__()
        self.efficientnet = models.efficientnet_b0(pretrained=False)  # Train from scratch
        self.efficientnet.classifier = nn.Sequential(
            nn.Linear(1280, 1),  # Binary classification
            nn.Sigmoid()
        )
    def forward(self, x):
        return self.efficientnet(x)

Initialize model, loss, and optimizer

In [13]:
model = EfficientNetBinaryClassifier().to(device)
criterion = nn.BCELoss()  # Binary Cross-Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.001)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


Training Function

In [14]:
def train(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, save_path="efficientnet_b0.pth"):
    best_val_accuracy = 0.0
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.float().to(device)

            # Forward pass
            outputs = model(images).view(-1)
            loss = criterion(outputs, labels)

            # Backward pass and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

        # Validation
        model.eval()
        val_loss = 0.0
        val_preds, val_labels = [], []
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.float().to(device)
                outputs = model(images).view(-1)
                loss = criterion(outputs, labels)
                val_loss += loss.item()
                val_preds.extend(outputs.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())

        # Calculate metrics
        val_preds = np.array(val_preds) > 0.5  # Convert probabilities to binary predictions
        val_accuracy = accuracy_score(val_labels, val_preds)
        val_roc_auc = roc_auc_score(val_labels, val_preds)
        val_f1 = f1_score(val_labels, val_preds)
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {running_loss/len(train_loader):.4f}, "
              f"Val Loss: {val_loss/len(val_loader):.4f}, "
              f"Val Accuracy: {val_accuracy:.4f}, "
              f"Val ROC-AUC: {val_roc_auc:.4f}, "
              f"Val F1-Score: {val_f1:.4f}")

        # Save the model if validation accuracy improves
        if val_accuracy > best_val_accuracy:
            best_val_accuracy = val_accuracy
            torch.save(model.state_dict(), save_path)
            print(f"Model saved to {save_path}")

Train the model

In [15]:
train(model, train_loader, val_loader, criterion, optimizer, num_epochs=100, save_path="efficientnet_b0.pth")

Epoch [1/1], Train Loss: 0.5500, Val Loss: 0.5829, Val Accuracy: 0.6667, Val ROC-AUC: 0.5000, Val F1-Score: 0.0000
Model saved to efficientnet_b0.pth


Test the model

In [16]:
def test(model, test_loader):
    model.eval()
    test_preds, test_labels = [], []
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.float().to(device)
            outputs = model(images).view(-1)
            test_preds.extend(outputs.cpu().numpy())
            test_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    test_preds = np.array(test_preds) > 0.5
    test_accuracy = accuracy_score(test_labels, test_preds)
    test_roc_auc = roc_auc_score(test_labels, test_preds)
    test_f1 = f1_score(test_labels, test_preds)
    print(f"Test Accuracy: {test_accuracy:.4f}, "
          f"Test ROC-AUC: {test_roc_auc:.4f}, "
          f"Test F1-Score: {test_f1:.4f}")

Evaluate on test set

In [17]:
test(model, test_loader)

Test Accuracy: 0.7416, Test ROC-AUC: 0.5000, Test F1-Score: 0.0000
